#Instalar libs

In [1]:
import pandas as pd
import numpy as np
from keras.layers import Input, Add
from keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

#Leitura dos arquivos disponibilizados

In [2]:
df = pd.read_csv('treino.csv',index_col=False, usecols=lambda x: x != 'Unnamed: 0')

In [3]:
df.head()

,Date,Close,Smoothed_Close,Label,Past_1_Days_Close,Past_2_Days_Close,Past_3_Days_Close,Past_4_Days_Close,Past_5_Days_Close,Past_6_Days_Close,Past_7_Days_Close,Past_8_Days_Close,Past_9_Days_Close,Past_10_Days_Close,Past_11_Days_Close,Past_12_Days_Close,Past_13_Days_Close,Past_14_Days_Close,Past_15_Days_Close
0,2000-01-24,1.409986,1.588954,-1,1.375888,1.478184,1.522513,1.566841,1.594120,1.500348,1.546381,1.704941,1.785073,1.863500,1.841336,1.853271,1.773139,1.682777,1.658908
1,2000-01-26,1.398052,1.569864,-1,1.409986,1.375888,1.478184,1.522513,1.566841,1.594120,1.500348,1.546381,1.704941,1.785073,1.863500,1.841336,1.853271,1.773139,1.682777
2,2000-01-27,1.375888,1.550466,-1,1.398052,1.409986,1.375888,1.478184,1.522513,1.566841,1.594120,1.500348,1.546381,1.704941,1.785073,1.863500,1.841336,1.853271,1.773139
3,2000-01-28,1.363953,1.531815,-1,1.375888,1.398052,1.409986,1.375888,1.478184,1.522513,1.566841,1.594120,1.500348,1.546381,1.704941,1.785073,1.863500,1.841336,1.853271
4,2000-01-31,1.329854,1.511619,-1,1.363953,1.375888,1.398052,1.409986,1.375888,1.478184,1.522513,1.566841,1.594120,1.500348,1.546381,1.704941,1.785073,1.863500,1.841336


In [4]:
df2 = pd.read_csv('teste.csv',index_col=False, usecols=lambda x: x != 'Unnamed: 0')

In [5]:
df2.head()

,Date,Close,Smoothed_Close,Label,Past_1_Days_Close,Past_2_Days_Close,Past_3_Days_Close,Past_4_Days_Close,Past_5_Days_Close,Past_6_Days_Close,Past_7_Days_Close,Past_8_Days_Close,Past_9_Days_Close,Past_10_Days_Close,Past_11_Days_Close,Past_12_Days_Close,Past_13_Days_Close,Past_14_Days_Close,Past_15_Days_Close
0,2019-03-25,9.917939,9.592937,1,9.957878,10.370573,10.583574,10.437135,10.849827,10.390541,10.250756,10.310663,9.432028,9.691624,9.159118,9.811440,10.084350,9.219026,8.706488
1,2019-03-26,10.543634,9.688007,1,9.917939,9.957878,10.370573,10.583574,10.437135,10.849827,10.390541,10.250756,10.310663,9.432028,9.691624,9.159118,9.811440,10.084350,9.219026
2,2019-03-27,10.004474,9.719654,1,10.543634,9.917939,9.957878,10.370573,10.583574,10.437135,10.849827,10.390541,10.250756,10.310663,9.432028,9.691624,9.159118,9.811440,10.084350
3,2019-03-28,10.437135,9.791402,1,10.004474,10.543634,9.917939,9.957878,10.370573,10.583574,10.437135,10.849827,10.390541,10.250756,10.310663,9.432028,9.691624,9.159118,9.811440
4,2019-03-29,10.816545,9.893916,1,10.437135,10.004474,10.543634,9.917939,9.957878,10.370573,10.583574,10.437135,10.849827,10.390541,10.250756,10.310663,9.432028,9.691624,9.159118


In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Close,4743.0,5.516489,3.691260,0.574565,2.911187,4.904093,6.975218,17.656988
Smoothed_Close,4743.0,5.501408,3.663616,0.634654,2.932332,4.955101,6.964744,16.677130
Label,4743.0,-0.119545,0.992934,-1.000000,-1.000000,-1.000000,1.000000,1.000000
Past_1_Days_Close,4743.0,5.514680,3.691186,0.574565,2.909560,4.902997,6.973883,17.656988
Past_2_Days_Close,4743.0,5.512805,3.690977,0.574565,2.907685,4.902997,6.966321,17.656988
Past_3_Days_Close,4743.0,5.510895,3.690697,0.574565,2.905375,4.899661,6.957801,17.656988
Past_4_Days_Close,4743.0,5.509024,3.690448,0.574565,2.903672,4.899661,6.955450,17.656988
Past_5_Days_Close,4743.0,5.507073,3.690070,0.574565,2.902880,4.899397,6.953801,17.656988
Past_6_Days_Close,4743.0,5.505199,3.689847,0.574565,2.900492,4.899397,6.953544,17.656988
Past_7_Days_Close,4743.0,5.503363,3.689651,0.574565,2.894990,4.897254,6.953051,17.656988


In [7]:
df['Date'].max()

'2019-03-22'

In [8]:
df['Date'].min()

'2000-01-24'

#Check para verificar os status da flag "Label" oferecida

In [9]:
df2['Label'][:100] ### Aqui é a tendencia de venda

# -1 para venda e 1 para compra


#Como é mais facil prever tendencia do que pelo proprio numero, vamos por aqui como target

0     1
1     1
2     1
3     1
4     1
     ..
95   -1
96   -1
97   -1
98   -1
99   -1
Name: Label, Length: 100, dtype: int64

In [10]:
df2['Label'].unique()

array([ 1, -1])

#Criação das bases de treino e teste, baseadas nos arquivos

In [11]:
x_train = df.drop(['Label','Smoothed_Close','Close'],axis=1 )

In [12]:
x_test = df2.drop(['Label','Smoothed_Close','Close'],axis=1)

In [13]:
y_train = df['Label']

In [14]:
y_test = df2['Label']

#Normalização das features e codificação

In [15]:
x_train.columns

Index(['Date', 'Past_1_Days_Close', 'Past_2_Days_Close', 'Past_3_Days_Close',
       'Past_4_Days_Close', 'Past_5_Days_Close', 'Past_6_Days_Close',
       'Past_7_Days_Close', 'Past_8_Days_Close', 'Past_9_Days_Close',
       'Past_10_Days_Close', 'Past_11_Days_Close', 'Past_12_Days_Close',
       'Past_13_Days_Close', 'Past_14_Days_Close', 'Past_15_Days_Close'],
      dtype='object')

In [16]:
# Normalizar os dados de Treino
scaler = MinMaxScaler(feature_range=(-1, 1))
lista_features = ['Past_1_Days_Close',
       'Past_2_Days_Close', 'Past_3_Days_Close', 'Past_4_Days_Close',
       'Past_5_Days_Close', 'Past_6_Days_Close', 'Past_7_Days_Close',
       'Past_8_Days_Close', 'Past_9_Days_Close', 'Past_10_Days_Close',
       'Past_11_Days_Close', 'Past_12_Days_Close', 'Past_13_Days_Close',
       'Past_14_Days_Close', 'Past_15_Days_Close']

x_train_scaled_data = pd.DataFrame(scaler.fit_transform(x_train[lista_features]),columns=lista_features)

In [17]:
x_train_scaled_data

,Past_1_Days_Close,Past_2_Days_Close,Past_3_Days_Close,Past_4_Days_Close,Past_5_Days_Close,Past_6_Days_Close,Past_7_Days_Close,Past_8_Days_Close,Past_9_Days_Close,Past_10_Days_Close,Past_11_Days_Close,Past_12_Days_Close,Past_13_Days_Close,Past_14_Days_Close,Past_15_Days_Close
0,-0.906182,-0.894205,-0.889015,-0.883825,-0.880631,-0.891610,-0.886220,-0.867656,-0.858274,-0.849092,-0.851687,-0.850290,-0.859672,-0.870251,-0.873046
1,-0.902189,-0.906182,-0.894205,-0.889015,-0.883825,-0.880631,-0.891610,-0.886220,-0.867656,-0.858274,-0.849092,-0.851687,-0.850290,-0.859672,-0.870251
2,-0.903587,-0.902189,-0.906182,-0.894205,-0.889015,-0.883825,-0.880631,-0.891610,-0.886220,-0.867656,-0.858274,-0.849092,-0.851687,-0.850290,-0.859672
3,-0.906182,-0.903587,-0.902189,-0.906182,-0.894205,-0.889015,-0.883825,-0.880631,-0.891610,-0.886220,-0.867656,-0.858274,-0.849092,-0.851687,-0.850290
4,-0.907579,-0.906182,-0.903587,-0.902189,-0.906182,-0.894205,-0.889015,-0.883825,-0.880631,-0.891610,-0.886220,-0.867656,-0.858274,-0.849092,-0.851687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4738,0.149249,0.132883,0.139897,0.037027,0.067420,0.005074,0.081448,0.113400,0.012088,-0.047919,-0.079092,-0.039347,-0.046361,-0.100913,-0.168714
4739,0.203022,0.149249,0.132883,0.139897,0.037027,0.067420,0.005074,0.081448,0.113400,0.012088,-0.047919,-0.079092,-0.039347,-0.046361,-0.100913
4740,0.154704,0.203022,0.149249,0.132883,0.139897,0.037027,0.067420,0.005074,0.081448,0.113400,0.012088,-0.047919,-0.079092,-0.039347,-0.046361
4741,0.171849,0.154704,0.203022,0.149249,0.132883,0.139897,0.037027,0.067420,0.005074,0.081448,0.113400,0.012088,-0.047919,-0.079092,-0.039347


In [18]:
# Converter a coluna 'Date' para o tipo datetime
x_train['Date'] = pd.to_datetime(df['Date'])

# Extrair o dia da semana (0 = segunda-feira, 1 = terça-feira, ..., 6 = domingo)
x_train['Day_of_week'] = x_train['Date'].dt.dayofweek

# One-hot encoding para o dia da semana
x_train = pd.get_dummies(x_train, columns=['Day_of_week'], prefix='Day')

# Extrair o mês
x_train['Month'] = x_train['Date'].dt.month

# One-hot encoding para o mês
x_train = pd.get_dummies(x_train, columns=['Month'], prefix='Month')

# Extrair o ano
x_train['Year'] = x_train['Date'].dt.year

# One-hot encoding para o ano
x_train = pd.get_dummies(x_train, columns=['Year'], prefix='Year')

# Remover a coluna 'Date', pois não é mais necessária
x_train.drop(columns=['Date'], inplace=True)

In [19]:
x_train.head()

,Past_1_Days_Close,Past_2_Days_Close,Past_3_Days_Close,Past_4_Days_Close,Past_5_Days_Close,Past_6_Days_Close,Past_7_Days_Close,Past_8_Days_Close,Past_9_Days_Close,Past_10_Days_Close,...,Year_2010,Year_2011,Year_2012,Year_2013,Year_2014,Year_2015,Year_2016,Year_2017,Year_2018,Year_2019
0,1.375888,1.478184,1.522513,1.566841,1.594120,1.500348,1.546381,1.704941,1.785073,1.863500,...,0,0,0,0,0,0,0,0,0,0
1,1.409986,1.375888,1.478184,1.522513,1.566841,1.594120,1.500348,1.546381,1.704941,1.785073,...,0,0,0,0,0,0,0,0,0,0
2,1.398052,1.409986,1.375888,1.478184,1.522513,1.566841,1.594120,1.500348,1.546381,1.704941,...,0,0,0,0,0,0,0,0,0,0
3,1.375888,1.398052,1.409986,1.375888,1.478184,1.522513,1.566841,1.594120,1.500348,1.546381,...,0,0,0,0,0,0,0,0,0,0
4,1.363953,1.375888,1.398052,1.409986,1.375888,1.478184,1.522513,1.566841,1.594120,1.500348,...,0,0,0,0,0,0,0,0,0,0


In [20]:
x_train.columns

Index(['Past_1_Days_Close', 'Past_2_Days_Close', 'Past_3_Days_Close',
       'Past_4_Days_Close', 'Past_5_Days_Close', 'Past_6_Days_Close',
       'Past_7_Days_Close', 'Past_8_Days_Close', 'Past_9_Days_Close',
       'Past_10_Days_Close', 'Past_11_Days_Close', 'Past_12_Days_Close',
       'Past_13_Days_Close', 'Past_14_Days_Close', 'Past_15_Days_Close',
       'Day_0', 'Day_1', 'Day_2', 'Day_3', 'Day_4', 'Month_1', 'Month_2',
       'Month_3', 'Month_4', 'Month_5', 'Month_6', 'Month_7', 'Month_8',
       'Month_9', 'Month_10', 'Month_11', 'Month_12', 'Year_2000', 'Year_2001',
       'Year_2002', 'Year_2003', 'Year_2004', 'Year_2005', 'Year_2006',
       'Year_2007', 'Year_2008', 'Year_2009', 'Year_2010', 'Year_2011',
       'Year_2012', 'Year_2013', 'Year_2014', 'Year_2015', 'Year_2016',
       'Year_2017', 'Year_2018', 'Year_2019'],
      dtype='object')

In [21]:
x_train_date =x_train.filter(regex='^(Day|Month|Year)', axis=1)

In [22]:
x_train_cod = pd.concat([x_train_date,x_train_scaled_data],axis=1)

In [23]:
x_train_cod

,Day_0,Day_1,Day_2,Day_3,Day_4,Month_1,Month_2,Month_3,Month_4,Month_5,...,Past_6_Days_Close,Past_7_Days_Close,Past_8_Days_Close,Past_9_Days_Close,Past_10_Days_Close,Past_11_Days_Close,Past_12_Days_Close,Past_13_Days_Close,Past_14_Days_Close,Past_15_Days_Close
0,1,0,0,0,0,1,0,0,0,0,...,-0.891610,-0.886220,-0.867656,-0.858274,-0.849092,-0.851687,-0.850290,-0.859672,-0.870251,-0.873046
1,0,0,1,0,0,1,0,0,0,0,...,-0.880631,-0.891610,-0.886220,-0.867656,-0.858274,-0.849092,-0.851687,-0.850290,-0.859672,-0.870251
2,0,0,0,1,0,1,0,0,0,0,...,-0.883825,-0.880631,-0.891610,-0.886220,-0.867656,-0.858274,-0.849092,-0.851687,-0.850290,-0.859672
3,0,0,0,0,1,1,0,0,0,0,...,-0.889015,-0.883825,-0.880631,-0.891610,-0.886220,-0.867656,-0.858274,-0.849092,-0.851687,-0.850290
4,1,0,0,0,0,1,0,0,0,0,...,-0.894205,-0.889015,-0.883825,-0.880631,-0.891610,-0.886220,-0.867656,-0.858274,-0.849092,-0.851687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4738,1,0,0,0,0,0,0,1,0,0,...,0.005074,0.081448,0.113400,0.012088,-0.047919,-0.079092,-0.039347,-0.046361,-0.100913,-0.168714
4739,0,1,0,0,0,0,0,1,0,0,...,0.067420,0.005074,0.081448,0.113400,0.012088,-0.047919,-0.079092,-0.039347,-0.046361,-0.100913
4740,0,0,1,0,0,0,0,1,0,0,...,0.037027,0.067420,0.005074,0.081448,0.113400,0.012088,-0.047919,-0.079092,-0.039347,-0.046361
4741,0,0,0,1,0,0,0,1,0,0,...,0.139897,0.037027,0.067420,0.005074,0.081448,0.113400,0.012088,-0.047919,-0.079092,-0.039347


In [24]:
# Normalizar os dados de teste
scaler = MinMaxScaler(feature_range=(-1, 1))
lista_features = ['Past_1_Days_Close',
       'Past_2_Days_Close', 'Past_3_Days_Close', 'Past_4_Days_Close',
       'Past_5_Days_Close', 'Past_6_Days_Close', 'Past_7_Days_Close',
       'Past_8_Days_Close', 'Past_9_Days_Close', 'Past_10_Days_Close',
       'Past_11_Days_Close', 'Past_12_Days_Close', 'Past_13_Days_Close',
       'Past_14_Days_Close', 'Past_15_Days_Close']

x_test_scaled_data = pd.DataFrame(scaler.fit_transform(x_test[lista_features]),columns=lista_features)

In [25]:
x_test_scaled_data

,Past_1_Days_Close,Past_2_Days_Close,Past_3_Days_Close,Past_4_Days_Close,Past_5_Days_Close,Past_6_Days_Close,Past_7_Days_Close,Past_8_Days_Close,Past_9_Days_Close,Past_10_Days_Close,Past_11_Days_Close,Past_12_Days_Close,Past_13_Days_Close,Past_14_Days_Close,Past_15_Days_Close
0,-0.676766,-0.653080,-0.640856,-0.649260,-0.625575,-0.651934,-0.659957,-0.656519,-0.706945,-0.692047,-0.722608,-0.685170,-0.669507,-0.719170,-0.748585
1,-0.679058,-0.676766,-0.653080,-0.640856,-0.649260,-0.625575,-0.651934,-0.659957,-0.656519,-0.706945,-0.692047,-0.722608,-0.685170,-0.669507,-0.719170
2,-0.643148,-0.679058,-0.676766,-0.653080,-0.640856,-0.649260,-0.625575,-0.651934,-0.659957,-0.656519,-0.706945,-0.692047,-0.722608,-0.685170,-0.669507
3,-0.674092,-0.643148,-0.679058,-0.676766,-0.653080,-0.640856,-0.649260,-0.625575,-0.651934,-0.659957,-0.656519,-0.706945,-0.692047,-0.722608,-0.685170
4,-0.649260,-0.674092,-0.643148,-0.679058,-0.676766,-0.653080,-0.640856,-0.649260,-0.625575,-0.651934,-0.659957,-0.656519,-0.706945,-0.692047,-0.722608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181,-0.193405,-0.175039,-0.184796,-0.213492,-0.237596,-0.242762,-0.263997,-0.264571,-0.261127,-0.274327,-0.265145,-0.262275,-0.272032,-0.263423,-0.305893
1182,-0.154952,-0.193405,-0.175039,-0.184796,-0.213492,-0.237596,-0.242762,-0.263997,-0.264571,-0.261127,-0.274327,-0.265145,-0.262275,-0.272032,-0.263423
1183,-0.151509,-0.154952,-0.193405,-0.175039,-0.184796,-0.213492,-0.237596,-0.242762,-0.263997,-0.264571,-0.261127,-0.274327,-0.265145,-0.262275,-0.272032
1184,-0.137161,-0.151509,-0.154952,-0.193405,-0.175039,-0.184796,-0.213492,-0.237596,-0.242762,-0.263997,-0.264571,-0.261127,-0.274327,-0.265145,-0.262275


In [26]:
# Converter a coluna 'Date' para o tipo datetime
x_test['Date'] = pd.to_datetime(x_test['Date'])

# Extrair o dia da semana (0 = segunda-feira, 1 = terça-feira, ..., 6 = domingo)
x_test['Day_of_week'] = x_test['Date'].dt.dayofweek

# One-hot encoding para o dia da semana
x_test = pd.get_dummies(x_test, columns=['Day_of_week'], prefix='Day')

# Extrair o mês
x_test['Month'] = x_test['Date'].dt.month

# One-hot encoding para o mês
x_test = pd.get_dummies(x_test, columns=['Month'], prefix='Month')

# Extrair o ano
x_test['Year'] = x_test['Date'].dt.year

# One-hot encoding para o ano
x_test = pd.get_dummies(x_test, columns=['Year'], prefix='Year')

# Remover a coluna 'Date', pois não é mais necessária
x_test.drop(columns=['Date'], inplace=True)

In [27]:
x_test.columns

Index(['Past_1_Days_Close', 'Past_2_Days_Close', 'Past_3_Days_Close',
       'Past_4_Days_Close', 'Past_5_Days_Close', 'Past_6_Days_Close',
       'Past_7_Days_Close', 'Past_8_Days_Close', 'Past_9_Days_Close',
       'Past_10_Days_Close', 'Past_11_Days_Close', 'Past_12_Days_Close',
       'Past_13_Days_Close', 'Past_14_Days_Close', 'Past_15_Days_Close',
       'Day_0', 'Day_1', 'Day_2', 'Day_3', 'Day_4', 'Month_1', 'Month_2',
       'Month_3', 'Month_4', 'Month_5', 'Month_6', 'Month_7', 'Month_8',
       'Month_9', 'Month_10', 'Month_11', 'Month_12', 'Year_2019', 'Year_2020',
       'Year_2021', 'Year_2022', 'Year_2023'],
      dtype='object')

In [28]:
x_test_date = x_test.filter(regex='^(Day|Month|Year)', axis=1)

In [29]:
x_test_date.columns

Index(['Day_0', 'Day_1', 'Day_2', 'Day_3', 'Day_4', 'Month_1', 'Month_2',
       'Month_3', 'Month_4', 'Month_5', 'Month_6', 'Month_7', 'Month_8',
       'Month_9', 'Month_10', 'Month_11', 'Month_12', 'Year_2019', 'Year_2020',
       'Year_2021', 'Year_2022', 'Year_2023'],
      dtype='object')

In [30]:
x_test_date

,Day_0,Day_1,Day_2,Day_3,Day_4,Month_1,Month_2,Month_3,Month_4,Month_5,...,Month_8,Month_9,Month_10,Month_11,Month_12,Year_2019,Year_2020,Year_2021,Year_2022,Year_2023
0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1182,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1183,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1184,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [31]:
# Normalização das colunas de base de treino e teste
# Usando a tecnica de one hot encoding é normal não ter a mesma quantidade de colunas entre treino e teste,
# porem vamos equalizar essa quantidade de colunas

Colunas_teste = set(list(x_test_date.columns))
Colunas_treino = set(list(x_train_date.columns))

Colunas_faltantes = Colunas_treino - Colunas_teste

In [32]:
#Setando todas as colunas faltantes para o valor zero
for col in Colunas_faltantes:
    x_test_date[col] = 0

<ipython-input-32-5776e28fe7eb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_date[col] = 0
<ipython-input-32-5776e28fe7eb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_date[col] = 0
<ipython-input-32-5776e28fe7eb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

In [33]:
x_test_date = x_test_date[Colunas_treino]

<ipython-input-33-d8d648350949>:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  x_test_date = x_test_date[Colunas_treino]


In [34]:
x_test_date.columns

Index(['Month_11', 'Year_2018', 'Month_12', 'Year_2019', 'Year_2011', 'Day_1',
       'Month_7', 'Month_5', 'Year_2008', 'Year_2014', 'Year_2001',
       'Year_2002', 'Year_2013', 'Year_2012', 'Month_10', 'Year_2007',
       'Year_2015', 'Year_2010', 'Year_2009', 'Month_3', 'Month_6',
       'Year_2005', 'Month_1', 'Month_9', 'Year_2004', 'Day_2', 'Month_4',
       'Day_3', 'Year_2016', 'Year_2017', 'Year_2000', 'Month_8', 'Day_0',
       'Year_2006', 'Year_2003', 'Month_2', 'Day_4'],
      dtype='object')

In [35]:
x_test_cod = pd.concat([x_test_date,x_test_scaled_data],axis=1)

#REDE LSTM //Ja configuravel para alterar por timesteps

In [36]:

# Reformular os dados de treinamento e teste para incluir a dimensão dos timesteps
# Supondo que cada sequência tenha 1 timestep
timesteps = 1 # 1 timestep porque leva a consideração da previsao de 1 a 1 dia, tendo features de modo diario
input_dim = x_train_cod.shape[1] // timesteps  # Número de características por timestep

x_train_reshaped = x_train_cod.values.reshape((x_train_cod.shape[0], timesteps, input_dim))
x_test_reshaped = x_test_cod.values.reshape((x_test_cod.shape[0], timesteps, input_dim))



In [37]:
# Definir a entrada
inputs = Input(shape=(timesteps, input_dim))

# Primeira camada LSTM
lstm1 = LSTM(units=32, return_sequences=True)(inputs)

# Segunda camada LSTM
lstm2 = LSTM(units=32, return_sequences=True)(lstm1)

# Adicionar a saída da primeira LSTM à saída da segunda LSTM
residual_output = Add()([lstm1, lstm2])

# Última camada LSTM
final_lstm_output = LSTM(units=64)(residual_output)

# Camada de saída
output = Dense(1, activation='sigmoid')(final_lstm_output)

In [38]:
# Criar o modelo
model = Model(inputs=inputs, outputs=output)


In [39]:
# Compilar o modelo e definir o otimizador, função de perda, etc.
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [40]:
# Treinar o modelo
model.fit(x_train_reshaped, y_train, epochs=100, batch_size=32)

Epoch 1/100
149/149 [==============================] - 7s 5ms/step - loss: -0.5382 - accuracy: 0.0255
Epoch 2/100
149/149 [==============================] - 1s 5ms/step - loss: -3.3263 - accuracy: 0.0327
Epoch 3/100
149/149 [==============================] - 1s 5ms/step - loss: -6.8954 - accuracy: 0.1320
Epoch 4/100
149/149 [==============================] - 1s 5ms/step - loss: -10.2636 - accuracy: 0.1636
Epoch 5/100
149/149 [==============================] - 1s 5ms/step - loss: -13.5992 - accuracy: 0.1626
Epoch 6/100
149/149 [==============================] - 1s 5ms/step - loss: -17.4363 - accuracy: 0.1489
Epoch 7/100
149/149 [==============================] - 1s 5ms/step - loss: -20.7908 - accuracy: 0.1339
Epoch 8/100
149/149 [==============================] - 1s 5ms/step - loss: -23.7940 - accuracy: 0.1238
Epoch 9/100
149/149 [==============================] - 1s 5ms/step - loss: -26.8452 - accuracy: 0.1124
Epoch 10/100
149/149 [==============================] - 1s 5ms/step - loss: 

#Apresentação dos resultados

In [41]:
# Predizer valores usando a base de teste
predictions_model = model.predict(x_test_reshaped)  # Fazer previsões no conjunto de teste

38/38 [==============================] - 1s 2ms/step


In [42]:
predictions_model

array([[0.9460851 ],
       [0.8847995 ],
       [0.        ],
       ...,
       [0.9819044 ],
       [0.9514464 ],
       [0.79069686]], dtype=float32)

In [43]:
# Converter as previsões para as classes reais (-1 para Venda, 1 para Compra)
predictions_class = np.array([1 if p > 0 else -1 for pred in predictions_model for p in pred.flatten()])

# Calcular a acurácia
accuracy = accuracy_score(y_test, predictions_class)
print("Acurácia:", accuracy)
print(" ")
print("-----------------------------------------------------------------")
print(" ")
# Calcular a matriz de confusão
conf_matrix = confusion_matrix(y_test, predictions_class)
print("Matriz de Confusão:")
print(conf_matrix)
print(" ")
print("-----------------------------------------------------------------")
print(" ")
# Calcular precision e recall
report = classification_report(y_test, predictions_class)
print("Relatório de Classificação:")
print(report)


Acurácia: 0.7310286677908938
 
-----------------------------------------------------------------
 
Matriz de Confusão:
[[481 207]
 [112 386]]
 
-----------------------------------------------------------------
 
Relatório de Classificação:
              precision    recall  f1-score   support

          -1       0.81      0.70      0.75       688
           1       0.65      0.78      0.71       498

    accuracy                           0.73      1186
   macro avg       0.73      0.74      0.73      1186
weighted avg       0.74      0.73      0.73      1186

